# N-gram Tutorial (Python 2)

# Building N-Gram models

In [137]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
# Load text from Project Gutenberg URL
import urllib2
#f = urllib2.urlopen("http://www.gutenberg.org/files/1342/1342-0.txt", 'r')
#txt = f.read()
#f.close()
txt=""
with open("/mnt/c/Users/gmanish/Dropbox/latest/openminds/slides/TextMining/n-gram models/1342-0.txt") as f:
    content = f.readlines()
for c in content:
    txt+=c

print len(txt), ',', txt[:50] , '...'

724725 , ﻿The Project Gutenberg EBook of Pride and Prejud ...


Great, now lets split into words into a big list, splitting on anything non-alphanumeric [A-Za-z0-9] (as well as punctuation) and forcing everything lowercase

In [2]:
import re
words = re.split('[^A-Za-z]+', txt.lower())
words = filter(None, words) # Remove empty strings
# Print length of list
print(len(words))

125901


## Sets
From this we can now generate N-grams, lets start with a 1-gram, basically the set of all the words

In [3]:
import sets
# Create set of all unique words, this throws away any information about frequency however
gram1 = set(words)
print len(gram1)
# Instead of printing all the elements in the set, create an iterator and print 20 elements only
gram1_iter = iter(gram1)
print [gram1_iter.next() for i in xrange(20)]

6530
['foul', 'four', 'woods', 'hanging', 'woody', 'looking', 'eligible', 'scold', 'lord', 'meadows', 'sinking', 'leisurely', 'bringing', 'disturb', 'recollections', 'wednesday', 'piling', 'persisted', 'succession', 'tired']


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: DeprecationWarning: the sets module is deprecated
  if __name__ == '__main__':


Lets try and get the 2-gram now, which is pairs of words. Let's have a quick look to see the last 10 and how they look.

In [4]:
# See the last 10 pairs
for i in xrange(len(words)-10, len(words)-1):
    print words[i], words[i+1]

subscribe to
to our
our email
email newsletter
newsletter to
to hear
hear about
about new
new ebooks


Okay, seems good, lets get all word pairs, and then generate a set of unique pairs from it

In [5]:
word_pairs = [(words[i], words[i+1]) for i in xrange(len(words)-1)]
print len(word_pairs)
gram2 = set(word_pairs)
print len(gram2)
# Print 20 elements from gram2
gram2_iter = iter(gram2)
print [gram2_iter.next() for i in xrange(20)]

125900
55640
[('her', 'taste'), ('every', 'kind'), ('best', 'friend'), ('soothed', 'but'), ('seemed', 'most'), ('fortune', 'it'), ('of', 'thanking'), ('near', 'she'), ('understand', 'from'), ('it', 'looks'), ('have', 'made'), ('lucas', 'he'), ('fail', 'him'), ('new', 'to'), ('nothing', 'but'), ('fearful', 'on'), ('to', 'wander'), ('write', 'rather'), ('of', 'studying'), ('interruption', 'from')]


## Frequency

Okay, that was fun, but this isn't enough, we need frequency if we want to have any sense of probabilities, which is what N-grams are about. Instead of using sets, lets create a dictionary with counts

In [6]:
gram1 = dict()

# Populate 1-gram dictionary
for word in words:
    if gram1.has_key(word):
        gram1[word] += 1
    else:
        gram1[word] = 1 # Start a new entry with 1 count since saw it for the first time.

# Turn into a list of (word, count) sorted by count from most to least
gram1 = sorted(gram1.items(), key=lambda (word, count): -count)

# Print top 20 most frequent words
print gram1[:20]

[('the', 4507), ('to', 4243), ('of', 3730), ('and', 3658), ('her', 2225), ('i', 2070), ('a', 2012), ('in', 1937), ('was', 1847), ('she', 1710), ('that', 1594), ('it', 1550), ('not', 1450), ('you', 1428), ('he', 1339), ('his', 1271), ('be', 1260), ('as', 1192), ('had', 1177), ('with', 1100)]


For Pride and Prejudice, the words 'the', 'to', 'of', and 'and' were the top four most common words. Sounds about right, not too interesting yet, lets see what happens with 2-grams.

In [7]:
gram2 = dict()

# Populate 2-gram dictionary
for i in xrange(len(words)-1):
    key = (words[i], words[i+1])
    if gram2.has_key(key):
        gram2[key] += 1
    else:
        gram2[key] = 1

# Turn into a list of (word, count) sorted by count from most to least
gram2 = sorted(gram2.items(), key=lambda (_, count): -count)

# Print top 20 most frequent words
print gram2[:20]

[(('of', 'the'), 491), (('to', 'be'), 445), (('in', 'the'), 397), (('i', 'am'), 303), (('mr', 'darcy'), 273), (('to', 'the'), 268), (('of', 'her'), 261), (('it', 'was'), 251), (('of', 'his'), 235), (('she', 'was'), 212), (('she', 'had'), 205), (('had', 'been'), 200), (('it', 'is'), 194), (('i', 'have'), 188), (('to', 'her'), 179), (('that', 'he'), 177), (('could', 'not'), 167), (('he', 'had'), 166), (('and', 'the'), 165), (('for', 'the'), 163)]


It looks like `"of the"` and `"to be"` are the top two most common 2-grams, sounds good. 

## Next word prediction using 2-gram models (max prob)
What can we do with this? Well lets see what happens if we take a random word from all the words, and build a sentence by just choosing the most common pair that has that word as it's start.

In [8]:
start_word = words[len(words)/4] #just took some random word
print start_word

delighted


Now in a loop, iterate through the frequency list (most frequent first) and see if it matches the first word in a pair, if so, the next word is the second element in the word pair, and continue with that word. Stop after N words or the list does not contain that word.

    *Note* : gram2 is a list that contains (key,value) where key is a word pair (first, second),
             so you need element[0][0] for first word and element [0][1] for second word

In [9]:
def get2GramSentence(word, n = 50):
    for i in xrange(n):
        print word,
        # Find Next word
        word = next((element[0][1] for element in gram2 if element[0][0] == word), None)
        if not word:
            break

word = start_word
print "Start word: %s" % word

print "2-gram sentence: \"",
get2GramSentence(word, 20)
print "\""

Start word: delighted
2-gram sentence: " delighted with the whole of the whole of the whole of the whole of the whole of the whole of "


It gets stuck in a loop pretty much straight away. Not very interesting, try out other words and see what happens.

In [10]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print "Start word: %s" % word

    print "2-gram sentence: \"",
    get2GramSentence(word, 20)
    print "\""

Start word: and
2-gram sentence: " and the whole of the whole of the whole of the whole of the whole of the whole of the "
Start word: he
2-gram sentence: " he had been so much as to be so much as to be so much as to be so much "
Start word: she
2-gram sentence: " she was not be so much as to be so much as to be so much as to be so "
Start word: when
2-gram sentence: " when she was not be so much as to be so much as to be so much as to be "
Start word: john
2-gram sentence: " john with the whole of the whole of the whole of the whole of the whole of the whole of "
Start word: never
2-gram sentence: " never be so much as to be so much as to be so much as to be so much as "
Start word: i
2-gram sentence: " i am sure i am sure i am sure i am sure i am sure i am sure i am "
Start word: how
2-gram sentence: " how much as to be so much as to be so much as to be so much as to be "


## Next word prediction using 2-gram models (Weighted random choice based on freq)
Lets randomly choose from the subset of all 2grams that matches the first word, using a weighted-probability based on counts.

In [11]:
import random
def weighted_choice(choices):
   total = sum(w for c, w in choices)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choices:
      if upto + w > r:
         return c
      upto += w
    
def get2GramSentenceRandom(word, n = 50):
    for i in xrange(n):
        print word,
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram2 if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]

In [12]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print "Start word: %s" % word

    print "2-gram sentence: \"",
    get2GramSentenceRandom(word, 20)
    print "\""

Start word: and
2-gram sentence: " and by venturing to lady i moved mr collins as a carriage drove along the way i were the matter "
Start word: he
2-gram sentence: " he could your education can i instantly to the colour there were but every attempt little inconvenience to conceal your "
Start word: she
2-gram sentence: " she was in his sister and was able to make mr darcy just as your respected mother s being useful "
Start word: when
2-gram sentence: " when i had she had been too much that they had been as she had done as i always coincide "
Start word: john
2-gram sentence: " john told what a clergyman i believe in the party distributing project gutenberg tm license included jane in what are "
Start word: never
2-gram sentence: " never attended her a reconciliation with them do not a liking which miss de bourgh only one most humiliating picture "
Start word: i
2-gram sentence: " i am sure i would most willingly have been so soon as wickham as it in time when the greater "
Start word

Those are starting to look like sentences!


Let's try a longer sentence

In [13]:
word = 'it'
print "Start word: %s" % word
print "2-gram sentence: \"",
get2GramSentenceRandom(word, 50)
print "\""

Start word: it
2-gram sentence: " it was marked her triumph very pretty and folly indeed if elizabeth will approve of elizabeth some time i never be levelled at longbourn on her colour but as soon grow afraid he would not likely to whisper for she tried to the longbourn a smile could not long seated "


Pretty cool, lets see what happens when we go to N-grams above 2.
## Creating Tri-grams and higher n-gram models
Okay, let's create a Ngram generator that can let us make ngrams of arbitrary sizes

In [24]:
def generateUpToNgram(n=1):
    grams=dict()
    for j in xrange(1,n+1):
        gram=dict()
        for i in xrange(len(words)-(j-1)):
            if j==1: #unigrams
                #gram is a dict of word to count
                key = tuple(words[i:i+1])
                if gram.has_key(key):
                    gram[key] += 1
                else:
                    gram[key] = 1
            else: #n-grams with n>1
                #gram is a dict of n-1 sized gram to (dict of nth word to freq)
                key = tuple(words[i:i+j-1])
                localDict=dict()
                if gram.has_key(key):
                    localDict=gram[key]
                    if localDict.has_key(words[i+j-1]):
                        localDict[words[i+j-1]]+=1
                    else:
                        localDict[words[i+j-1]]=1
                else:
                    localDict[words[i+j-1]]=1
                gram[key] = localDict
        if j==1:
            gram = sorted(gram.items(), key=lambda (_, count): -count)
        if j!=1:
            for x in gram:
                gram[x] = sorted(gram[x].items(), key=lambda (_, count): -count)
        grams[j]=gram
    return grams

ngrams= generateUpToNgram(4)

In [62]:
# Print top few most frequent ngrams
def printNgrams(toprint, elementsToPrint):
    for i in toprint:
        print i
        k=0
        for j in toprint[i]:
            k+=1
            if i==1:
                print j
            else:
                print j, toprint[i][j]
            if k>elementsToPrint:
                break
printNgrams(ngrams,10)

1
(('the',), 4507)
(('to',), 4243)
(('of',), 3730)
(('and',), 3658)
(('her',), 2225)
(('i',), 2070)
(('a',), 2012)
(('in',), 1937)
(('was',), 1847)
(('she',), 1710)
(('that',), 1594)
2
('resolve',) [('very', 1), ('to', 1)]
('waited',) [('on', 3), ('for', 2), ('only', 1), ('at', 1)]
('housekeeping',) [('i', 1), ('when', 1), ('her', 1)]
('defective',) [('the', 1), ('work', 1), ('or', 1), ('you', 1)]
('intricate',) [('character', 1), ('characters', 1)]
('wavering',) [('before', 1)]
('now',) [('i', 9), ('and', 8), ('to', 6), ('be', 6), ('have', 5), ('as', 4), ('when', 4), ('a', 4), ('the', 4), ('she', 3), ('come', 3), ('made', 3), ('in', 3), ('indeed', 2), ('only', 2), ('do', 2), ('because', 2), ('absolutely', 2), ('said', 2), ('for', 2), ('told', 2), ('we', 2), ('put', 2), ('on', 2), ('of', 2), ('fast', 2), ('been', 2), ('passed', 2), ('was', 2), ('but', 2), ('convinced', 2), ('determined', 2), ('it', 2), ('at', 2), ('saw', 2), ('began', 2), ('so', 2), ('despise', 1), ('being', 1), ('open

# Generating text using n-gram models with n>=3

Cool! Okay, let's see a selection of sentences for N-grams with N = 2 to 10 and a few starting words!

In [63]:
def getUpToNGramSentenceMax(gram, word, n = 50):
    #hist is a window of previous few words
    hist=list()
    maxHistorySize=len(gram)-1
    for i in xrange(n):
        if len(hist)>=maxHistorySize:
            hist.pop(0)
        hist.append(word)
        found=0
        for j in xrange(len(hist)): #start with longest possible history
            key=tuple(hist[j:])
            choices = [gram[len(key)+1][element] for element in gram[len(key)+1] if element == key]
            if choices:
                found=1
                break
        if found==0:
            break
        word=choices[0][0][0]
        print word,

for n in xrange(2,10):
    print
    print "Generating %d-gram list..." % n,
    ngram=generateUpToNgram(n)
    print "Done"
    # Try out a bunch of sentences
    for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
        print "  %d-gram: \"" % n, word,
        getUpToNGramSentenceMax(ngram, word, 15)
        print "\""


Generating 2-gram list... Done
  2-gram: " and the whole party were to be so much as to be so much as to "
  2-gram: " he had been so much as to be so much as to be so much as "
  2-gram: " she was not be so much as to be so much as to be so much "
  2-gram: " when she was not be so much as to be so much as to be so "
  2-gram: " john with the whole party were to be so much as to be so much as "
  2-gram: " never be so much as to be so much as to be so much as to "
  2-gram: " i am sure i am sure i am sure i am sure i am sure i "
  2-gram: " how much as to be so much as to be so much as to be so "

Generating 3-gram list... Done
  3-gram: " and the other day we had better have taken you to be in london and last "
  3-gram: " he had been so fortunate as to the project gutenberg tm electronic works in creating the "
  3-gram: " she was not in the world and everybody said how pleasant it is not to be "
  3-gram: " when she had been so fortunate as to the project gutenberg tm electronic w

In [64]:
def getUpToNGramSentenceRandom(gram, word, n = 50):
    hist=list()
    maxHistorySize=len(gram)-1
    for i in xrange(n):
        if len(hist)>=maxHistorySize:
            hist.pop(0)
        hist.append(word)
        found=0
        for j in xrange(len(hist)):
            key=tuple(hist[j:])
            choices = [gram[len(key)+1][element] for element in gram[len(key)+1] if element == key]
            if choices:
                found=1
                break
        if found==0:
            break
        word = weighted_choice(choices[0])
        print word,

for n in xrange(2,10):
    print
    print "Generating %d-gram list..." % n,
    ngram=generateUpToNgram(n)
    print "Done"
    # Try out a bunch of sentences
    for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
        print "  %d-gram: \"" % n, word,
        getUpToNGramSentenceRandom(ngram, word, 15)
        print "\""


Generating 2-gram list... Done
  2-gram: " and indifferent indeed not know what do not time but this agreement you to be at "
  2-gram: " he went out the family they had mentioned his judgement controverted she could be highly esteemed "
  2-gram: " she added may be generally bestowed my going away as my side of appearing on you "
  2-gram: " when he must be her and prefer a countenance such as well is so odd a "
  2-gram: " john with pleasure in nothing escaped his wife s marriage of about his sisters behind he "
  2-gram: " never in all means capital one of seeing her excessive commendation of fashion so very proper "
  2-gram: " i should like to give me i have been in compliment to let me no scruple "
  2-gram: " how much beauty but there be doubted whether they could ever be supposed marriage he had "

Generating 3-gram list... Done
  3-gram: " and employees expend considerable effort much paperwork and many inquiries to make him marry her had "
  3-gram: " he was totally ignorant

You can clearly see the sentences getting better and better with larger n-grams, this correlates to the ngram having more foresight into the sentence structure.

In [65]:
# Generate 50gram list
n = 50
print
print "Generating %d-gram list..." % n,
gram50 = generateUpToNgram(n)
print "Done"


Generating 50-gram list... Done


In [66]:
# Try out a bunch of sentences
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print "  %d-gram: \"" % n,
    getUpToNGramSentenceRandom(gram50, word, 100)
    print "\""

  50-gram: " she only wanted to know how far she wished that welfare to depend upon herself and how far it would be for the happiness of both that she should employ the power which her fancy told her she still possessed of bringing on her the renewal of his addresses it had been settled in the evening between the aunt and the niece that such a striking civility as miss darcy s in coming to see them on the very day of her arrival at pemberley for she had reached it only to a late breakfast ought to be imitated though "
  50-gram: " never failed coming to inform them of though it happened almost every day she not unfrequently stopped at the parsonage and had a few minutes conversation with charlotte but was scarcely ever prevailed upon to get out very few days passed in which mr collins did not walk to rosings and not many in which his wife did not think it necessary to go likewise and till elizabeth recollected that there might be other family livings to be disposed of she could not und

# Laplace Smoothed n-grams

In [79]:
def getLaplaceSmoothedUpToNGramModel(n=2, printModels=True):
    # Generate ngram list
    print
    print "Generating %d-gram list...\n" % n,
    ngram = generateUpToNgram(n)
    if printModels:
        printNgrams(ngram,10)
    smoothed=dict()
    total=0
    for i in xrange(1,n+1):
        grams=ngram[i]
        smoothedGrams=dict()
        if i==1:
            total=0
            for key, val in grams:
                total+=val
            #print(total)
            for key, val in grams:
                smoothedGrams[key]=(float(val)+1)/(float(total)+len(grams))
            smoothedGrams = sorted(smoothedGrams.items(), key=lambda (_, count): -count)
        else:
            for key in grams:
                smoothedLocalDict=dict()
                localDict=grams[key]
                total=0
                for k,v in localDict:
                    total+=v
                for k,v in localDict:
                    smoothedLocalDict[k]=(float(v)+1)/(float(total)+len(ngram[1])) # ngram[1] because it defines the vocabulary
                smoothedLocalDict = sorted(smoothedLocalDict.items(), key=lambda (_, count): -count)
                smoothedGrams[key]=smoothedLocalDict
        smoothed[i]=smoothedGrams
    if printModels:
        printNgrams(smoothed, 10)
    return smoothed

_=getLaplaceSmoothedUpToNGramModel(2)


Generating 2-gram list...
1
(('the',), 4507)
(('to',), 4243)
(('of',), 3730)
(('and',), 3658)
(('her',), 2225)
(('i',), 2070)
(('a',), 2012)
(('in',), 1937)
(('was',), 1847)
(('she',), 1710)
(('that',), 1594)
2
('resolve',) [('very', 1), ('to', 1)]
('waited',) [('on', 3), ('for', 2), ('only', 1), ('at', 1)]
('housekeeping',) [('i', 1), ('when', 1), ('her', 1)]
('defective',) [('the', 1), ('work', 1), ('or', 1), ('you', 1)]
('intricate',) [('character', 1), ('characters', 1)]
('wavering',) [('before', 1)]
('now',) [('i', 9), ('and', 8), ('to', 6), ('be', 6), ('have', 5), ('as', 4), ('when', 4), ('a', 4), ('the', 4), ('she', 3), ('come', 3), ('made', 3), ('in', 3), ('indeed', 2), ('only', 2), ('do', 2), ('because', 2), ('absolutely', 2), ('said', 2), ('for', 2), ('told', 2), ('we', 2), ('put', 2), ('on', 2), ('of', 2), ('fast', 2), ('been', 2), ('passed', 2), ('was', 2), ('but', 2), ('convinced', 2), ('determined', 2), ('it', 2), ('at', 2), ('saw', 2), ('began', 2), ('so', 2), ('despise

# Computing perplexity

In [90]:
ngram = getLaplaceSmoothedUpToNGramModel(4, False)
#printNgrams(ngram,10)


Generating 4-gram list...


In [146]:
import math
def computePerplexity(model, sentence, order):
    toks=sentence.split(' ')
    logprob=0.0
    if order==1:
        for t in toks:
            val=dict(model[1])
            logprob+=math.log(val[tuple([t])])
        #print logprob
    #modelSize=len(next(iter(model)))+1
    else:
        for i in xrange(0, len(toks)-order+1):
            l=list()
            for j in xrange(i, i+order):
                print toks[j],
            print
            for j in xrange(i, i+order-1):
                l.append(toks[j])
            lastword=toks[i+order-1]
            print l, lastword
            val=dict(model[order])
            #print(dict(val[tuple(l)])[lastword])
            logprob+=math.log(dict(val[tuple(l)])[lastword])
            logprob/=order
            
    return -logprob
    
computePerplexity(ngram, "and i will", 1)
computePerplexity(ngram, "and will i", 1)
computePerplexity(ngram, "and i will", 2)
computePerplexity(ngram, "and will i", 2)


13.49334689669374

13.49334689669374

and i
['and'] i
i will
['i'] will


9.399726570492227

and will
['and'] will
will i
['will'] i


14.338080884761208